In [ ]:
!pip install selenium webdriver-manager

In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import UnexpectedAlertPresentException
import re
import time


options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


try:
    driver.get("https://gcloud.csi.go.kr/cmq/main.do")
    print("\n" + "="*70)
    print(" [작업 지시]")
    print(" 1. 로그인을 완료하고 상세 페이지를 띄우세요.")
    print(" 2. 페이지 로딩이 완료되면 여기서 Enter를 누르세요.")
    print("="*70)
    input(">> Enter를 누르면 엑셀 저장을 시작합니다: ")

    # 엑셀에 담을 데이터 리스트
    excel_data = []

    print("\n" + "="*70)
    print(" [데이터 추출 및 엑셀 정리 시작]")
    print("="*70)

    # ---------------------------------------------------------
    # [1] 하단부: 인적 사항 + 공사개요 + 기타 항목 (검증 완료 로직)
    # ---------------------------------------------------------
    
    # 1-1. 인적 사항
    print("\n[1. 인적 사항 정보 수집 중...]")
    for s in ["pick_user_text", "seal_user_text", "attnd_user_text"]:
        try:
            row = driver.find_element(By.XPATH, f"//th[@data-name='{s}']/..")
            val = row.text.strip().replace('\n', ' | ')
            print(f"> {val}")
            excel_data.append(["1.인적사항", s, val])
        except: continue

    # 1-2. 공사개요
    print("\n[2. 공사개요 상세 수집 중...]")
    c_fields = {"공사명": "bizNm", "현장소재지": "addr", "착공일": "bgngDt", "준공예정일": "cmplPlanDt"}
    for label, eid in c_fields.items():
        try:
            el = driver.find_element(By.ID, eid)
            val = el.get_attribute('value') or el.get_attribute('innerText').strip() or el.text.strip()
            print(f"> {label} : {val}")
            excel_data.append(["2.공사개요", label, val])
        except: continue

    # 1-3. 기타 항목
    print("\n[3. 기타 항목 수집 중...]")
    for label in ["성과 이용 목적", "발주자", "시공자", "국가중요시설 여부"]:
        try:
            val = driver.find_element(By.XPATH, f"//th[contains(text(), '{label}')]/following-sibling::td").text.strip()
            print(f"> {label} : {val}")
            excel_data.append(["3.기타항목", label, val])
        except: continue

    # ---------------------------------------------------------
    # [2] 상단부: 과거 성공 로직 (전체 테이블 스캔)
    # ---------------------------------------------------------
    print("\n[4. 상단 기본 정보 수집 중...]")
    try:
        top_container = driver.find_element(By.ID, "rqstViewAjax")
        top_tables = top_container.find_elements(By.TAG_NAME, "table")
        
        for table in top_tables:
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                text = row.get_attribute('innerText').strip().replace('\n', ' | ')
                if text:
                    print(f"  {text}")
                    excel_data.append(["4.상단정보", "상세내용", text])
    except Exception as e:
        print(f"> 상단부 출력 중 참고사항: {e}")

    # ---------------------------------------------------------
    # [3] 엑셀 파일 저장
    # ---------------------------------------------------------
    df = pd.DataFrame(excel_data, columns=["구분", "항목명", "내용"])
    file_name = "품질시험의뢰서_최종결과.xlsx"
    df.to_excel(file_name, index=False)

    print("\n" + "="*70)
    print(f" 성공! 모든 데이터가 '{file_name}' 파일로 저장되었습니다.")
    print(" 폴더에서 엑셀 파일을 열어 확인해 보세요.")
    print("="*70)

except Exception as e:
    print(f"\n[오류 발생] {e}")




 [작업 지시]
 1. 로그인을 완료하고 상세 페이지를 띄우세요.
 2. 페이지 로딩이 완료되면 여기서 Enter를 누르세요.

 [데이터 추출 및 엑셀 정리 시작]

[1. 인적 사항 정보 수집 중...]
> 채취자 소속 | 현대엔지니어링 주식회사 | 담당업무 | 품질관리 | 성명 | 정광훈 (서명 완료)

[2. 공사개요 상세 수집 중...]
> 공사명 : 부천대장 공종주택지구 조성공사 2공구
> 현장소재지 : 경기도 부천시 오정구 벌말로 43
> 착공일 : 2024-04-25

[3. 기타 항목 수집 중...]
> 성과 이용 목적 : 품질시험·검사 대행
> 발주자 : 한국토지주택공사 인천지역본부 (담당자: 최기찬)
> 시공자 : 현대엔지니어링 주식회사
> 국가중요시설 여부 : 해당없음

[4. 상단 기본 정보 수집 중...]
  의뢰번호	RQ-2025-244246	접수번호	AC-2025-330149	최종진행상태	시험·검사 진행 확정
  품질검사 대행기관	한국건설품질시험원 주식회사
  시험실명 및 주소	한국건설품질시험원 (주소:경기도 고양시 일산서구 송포백송길 47 (덕이동))	시험실 연락처	031-946-5518
  의뢰기관	현대엔지니어링 주식회사	의뢰인 | (의뢰인 번호)	정광훈 | (02--)	의뢰일시	2025-12-30 10:35:32
  입찰공고 마감일시	2025-12-30 12:00	의뢰 확인자	심억	입찰 공고 일시	2025-12-30 10:38:56
  접수일시	2025-12-30 13:09:02	시험·검사진행 | 확정일시	2025-12-30 13:09:02	성적서 발급 | 예정일자	2026-02-09 이력
  봉인명	원심력철근콘크리트관(D1100)	봉인번호	0000000430558
  시료정보	봉인상태	 | 시료정보 테이블입니다. | 시료명	시료량	생산자	채취일	채취장소	규격	가공상태 | 원심력철근콘크리트관(D1100) (S1)	1본	대일콘크리트(주)	2025-12-29	대일콘크리트 생산공장	D1100*2.5M	가공
  시료명	시료량	생산자